Os *tools* são componentes ou funcionalidades específicas que os agentes (AI models) podem usar para realizar ações ou interagir com outros sistemas. Esses tools permitem que o modelo de linguagem execute tarefas específicas, como buscar informações em uma API, consultar bancos de dados, realizar cálculos, ou até mesmo controlar o fluxo de uma conversa.

In [2]:
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field
import requests
import statistics

In [3]:
class DadosAcaoArgs(BaseModel):
    simbolo: str = Field(description='Símbolo da ação a ser buscada', examples=['AAPL', 'MSFT'])
    chave_api: str = Field(description='WE604VQLXNV37CEK')

@tool(args_schema=DadosAcaoArgs)
def buscar_e_analisar_dados_acao(simbolo: str, chave_api: str):
    '''Busca dados de ações na API Alpha Vantage e realiza uma análise simples'''
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={simbolo}&apikey={chave_api}'
    resposta = requests.get(url)
    dados = resposta.json()
    
    if 'Time Series (Daily)' not in dados:
        return 'Erro ao buscar dados da Alpha Vantage'
    
    serie_temporal = dados['Time Series (Daily)']
    
    precos_fechamento = [float(detalhes['4. close']) for data, detalhes in serie_temporal.items()]
    
    media_precos_fechamento = statistics.mean(precos_fechamento)
    
    return f'A média dos preços de fechamento para {simbolo} é {media_precos_fechamento:.2f}'

In [4]:
chave_api = 'WE604VQLXNV37CEK'
simbolo = 'AAPL'

args = DadosAcaoArgs(simbolo=simbolo, chave_api=chave_api)

resultado = buscar_e_analisar_dados_acao.invoke(args.dict())
print(resultado)

A média dos preços de fechamento para AAPL é 205.76
